In [ ]:
#def unpairedttest(ar1,ar2):
#    npar1 = np.array(ar1)
#    npar2 = np.array(ar2)
#    return np.abs(npar1.mean()-npar2.mean())/np.sqrt((npar1.var()/npar1.shape[0])+(npar2.var()/npar2.shape[0]))

In [1]:
from model import UNet
import helper_functions as hp
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.simplefilter("ignore")
import numpy as np
import os
import scipy.io
import json
from skimage import metrics
import scipy.stats

In [2]:
def pixelwiseFPFNTP(pred,gold):
    TP = np.where(gold+pred==2,1,0).sum()
    FP = np.where(pred-gold==1,1,0).sum()
    FN = np.where(gold-pred==1,1,0).sum()

    return TP, FP, FN

def getPrecRecallFScore(TP, FP, FN):
    if (TP + FP) > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    if (TP + FN) > 0:    
        recall = TP / (TP + FN)
    else:
        recall = 0
    if (precision+recall) > 0:    
        f_score = (2*precision*recall)/(precision+recall)
    else:
        f_score = 0
        
    return precision, recall, f_score 

def pixelwise(pred,gold):
    
    TP, FP, FN = pixelwiseFPFNTP(pred,gold)

    return getPrecRecallFScore(TP, FP, FN)

def IoU(prediction, goldB, threshold):
    goldB_cc = cv2.connectedComponents(goldB)
    pred_cc = cv2.connectedComponents(prediction)
    TP = 0
    for i in range(1,goldB_cc[0]+1):
        goldB_inst = np.where(goldB_cc[1]==i,1,0)
        
        inter_num = 0
        for j in range(1,pred_cc[0]+1):
            pred_inst = np.where(pred_cc[1]==j,1,0)
            union = goldB_inst + pred_inst
            if union.max() == 2:
                intersection = np.where(union==2,1,0)
                inter_num_curr = intersection.sum()
                if inter_num_curr > inter_num:
                    inter_num = inter_num_curr
                    union_chosen = union
                    
        if inter_num > 0:
            union_chosen = np.where(union_chosen>0,1,0)
            union_num = union_chosen.sum()
            if inter_num/union_num >= threshold:
                TP = TP + 1
                
    segmented = pred_cc[0] - 1
    positives = goldB_cc[0] - 1
    if segmented > 0:
        precision = TP / segmented
    else:
        precision = 0
    recall = TP / positives
    if precision+recall > 0:
        f_score = round(2*((precision*recall)/(precision+recall)),2)
    else:
        f_score = 0      
        
    return precision, recall, f_score

def get_weighted_hausdorff_dist(pred,gold):
    flag = False;
    if pred.sum() == 0:
        pred[0,:] = 1; pred[255,:] = 1; pred[:,0] = 1; pred[:,255] = 1;  
        flag = True
    gold_cc = cv2.connectedComponents(gold)
    pred_cc = cv2.connectedComponents(pred)
    return 0.5 * (wdh_one(pred_cc, gold_cc, pred,gold, flag) + wdh_one(gold_cc, pred_cc, gold,pred,flag))

def wdh_one(first_cc,second_cc,first,second, flag):
    whd = 0
    for i in range(1,first_cc[0]):
        first_inst = np.where(first_cc[1]==i,1,0)

        weight = first_inst.sum() / first.sum()

        if (first_inst * second).sum() > 0:
            intersection = 0
            for j in range(1,second_cc[0]):
                second_inst = np.where(second_cc[1]==j,1,0)
                intersection_new = (first_inst * second_inst).sum()
                if intersection_new > intersection:
                    hd = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    intersection = intersection_new
        else:
            if flag:
                hd = 0
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new > hd:
                        hd = hd_new                  
            else:
                hd = np.inf
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new < hd:
                        hd = hd_new  
    
        whd = whd + (weight * hd)   
    return whd        

In [3]:
first_out_channel = 32
gold_folder = 'data/goldsBinaryAll'
mn_dice = ['Dice/weights/allWeightsB2.pth','Dice/weights/allWeightsB2_2.pth','Dice/weights/allWeightsB2_3.pth','Dice/weights/allWeightsB2_4.pth','Dice/weights/allWeightsB2_5.pth']
mn_dice_our = ['DiceOur/weights/allWeightsB2.pth','DiceOur/weights/allWeightsB2_2.pth','DiceOur/weights/allWeightsB2_3.pth','DiceOur/weights/allWeightsB2_4.pth','DiceOur/weights/allWeightsB2_5.pth']

In [5]:
import os
import json

with open('data/vesseltypes.json') as f:
    fileTypesDict = json.load(f)
    
image_names = os.listdir('data/bucket2/ts')

typ = []

dicePrecisionPixel = []
diceRecallPixel = []
diceFscorePixel = []
diceOurPrecisionPixel = []
diceOurRecallPixel = []
diceOurFscorePixel = []
diceHaus = []
diceOurHaus = []


for i in range(5):
    for name in image_names:    
        typ.append(fileTypesDict[name])

        tst_im_name = 'data/bucket2/ts/' + name
        gold_im_name = 'data/goldsBinaryAll/' + name.split('.')[0] + '.png'

        _, _, pred_base, gold = hp.test(1, first_out_channel, mn_dice[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_base, gold)
        dicePrecisionPixel.append(precision)
        diceRecallPixel.append(recall)
        diceFscorePixel.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_base, gold)
        diceHaus.append(whd)

        _, _, pred_our, _ = hp.test(1, first_out_channel, mn_dice_our[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_our, gold)
        diceOurPrecisionPixel.append(precision)
        diceOurRecallPixel.append(recall)
        diceOurFscorePixel.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_our, gold)
        diceOurHaus.append(whd)

In [6]:
def pairedttest(ar1,ar2):#we should use this because we take measurement on prediction
    arr = np.array(ar1) - np.array(ar2)
    return arr.mean()/(arr.std()/np.sqrt(arr.shape[0])),np.array(ar1).mean(),np.array(ar2).mean()

In [8]:
compare_list = [['diceOurPrecisionPixel',diceOurPrecisionPixel, dicePrecisionPixel],
['diceOurRecallPixel',diceOurRecallPixel, diceRecallPixel],
['diceOurFscorePixel',diceOurFscorePixel, diceFscorePixel],
['diceOurHaus',diceOurHaus, diceHaus]]

In [9]:
for item in compare_list:
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] == 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] == 'f']
    print(item[0], 'big')
    print(pairedttest(ar1, ar2))
    print(scipy.stats.ttest_rel(ar1, ar2))
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] != 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] != 'f']
    print(item[0], 'other')
    print(pairedttest(ar1, ar2))
    print(scipy.stats.ttest_rel(ar1, ar2))
    print(item[0], 'all')
    print(pairedttest(item[1], item[2]))
    print(scipy.stats.ttest_rel(item[1], item[2]))    

diceOurPrecisionPixel big
(-6.609983876500896, 0.9198802407080995, 0.9257979695392933)
TtestResult(statistic=-6.609021654517997, pvalue=4.467821085623199e-11, df=3434)
diceOurPrecisionPixel other
(4.194046827180922, 0.8199710956315317, 0.8123815058683699)
TtestResult(statistic=4.193483073320296, pvalue=2.811050051025351e-05, df=3719)
diceOurPrecisionPixel all
(1.0650650942756912, 0.8679358633936575, 0.8668309192449769)
TtestResult(statistic=1.064990663645999, pvalue=0.28691609720528743, df=7154)
diceOurRecallPixel big
(7.892872786231598, 0.9298839337290968, 0.9194936365564239)
TtestResult(statistic=7.8917238128233365, pvalue=3.974860230140564e-15, df=3434)
diceOurRecallPixel other
(4.281089063178595, 0.8455657453045812, 0.8367007416058548)
TtestResult(statistic=4.2805136093063565, pvalue=1.9116470576926948e-05, df=3719)
diceOurRecallPixel all
(7.6874702073199925, 0.8860455464559734, 0.8764482739825432)
TtestResult(statistic=7.686932978890159, pvalue=1.70842902727742e-14, df=7154)
diceO

In [10]:
def IoU_accm(prediction, goldB, threshold):
    goldB_cc = cv2.connectedComponents(goldB)
    pred_cc = cv2.connectedComponents(prediction)
    TP = 0
    for i in range(1,goldB_cc[0]+1):
        goldB_inst = np.where(goldB_cc[1]==i,1,0)
        
        inter_num = 0
        for j in range(1,pred_cc[0]+1):
            pred_inst = np.where(pred_cc[1]==j,1,0)
            union = goldB_inst + pred_inst
            if union.max() == 2:
                intersection = np.where(union==2,1,0)
                inter_num_curr = intersection.sum()
                if inter_num_curr > inter_num:
                    inter_num = inter_num_curr
                    union_chosen = union
                    
        if inter_num > 0:
            union_chosen = np.where(union_chosen>0,1,0)
            union_num = union_chosen.sum()
            if inter_num/union_num >= threshold:
                TP = TP + 1
                
    segmented = pred_cc[0] - 1
    positives = goldB_cc[0] - 1     
        
    return segmented, positives, TP

def getPrecRecallFScore_IoU_accm(segmented,positives,TP):
    precision = TP/segmented
    recall = TP/positives
    f_score = 2*((precision*recall)/(precision+recall))
    return precision, recall, f_score

def getResultsIoU_accm(patient, model_name):
    
    with open('data/vesseltypes.json') as f:
        fileTypesDict = json.load(f)
            
    image_names = [im_name for im_name in os.listdir('data/bucket2/ts') if im_name.split('_')[0] == patient]
        
    segmented_sum_f = 0
    positives_sum_f = 0
    TP_sum_f = 0
    segmented_sum_s = 0
    positives_sum_s = 0
    TP_sum_s = 0
                
    for name in image_names:
        tst_im_name = 'data/bucket2/ts/' + name
        gold_im_name = 'data/goldsBinaryAll/' + name.split('.')[0] + '.png'
        
        if model_name[:7] == 'Fourier':
            prediction = cv2.imread(model_name + name.split('.')[0] + '.png',cv2.IMREAD_GRAYSCALE)
            goldB = cv2.imread(gold_im_name,cv2.IMREAD_GRAYSCALE)
        else:    
            _, _, prediction, goldB = hp.test(1, 32, model_name, tst_im_name, gold_im_name)

        segmented, positives, TP = IoU_accm(prediction, goldB, 0.5)

        if fileTypesDict[name] == 'f':
            segmented_sum_f = segmented_sum_f + segmented
            positives_sum_f = positives_sum_f + positives
            TP_sum_f = TP_sum_f + TP
        else:
            segmented_sum_s = segmented_sum_s + segmented
            positives_sum_s = positives_sum_s + positives
            TP_sum_s = TP_sum_s + TP
    
    segmented_sum = segmented_sum_f + segmented_sum_s
    positives_sum = positives_sum_f + positives_sum_s
    TP_sum = TP_sum_f + TP_sum_s
    results = [0,0,0,0,0,0,0,0,0]
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum,positives_sum,TP_sum)    
    results[0] = round(precision,4)
    results[1] = round(recall,4)
    results[2] = round(f_score,4)
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum_f,positives_sum_f,TP_sum_f)    
    results[3] = round(precision,4)
    results[4] = round(recall,4)
    results[5] = round(f_score,4)
    
    precision, recall, f_score = getPrecRecallFScore_IoU_accm(segmented_sum_s,positives_sum_s,TP_sum_s)    
    results[6] = round(precision,4)
    results[7] = round(recall,4)
    results[8] = round(f_score,4)  
    
    return results

In [11]:
patients = np.unique([p_im.split('_')[0] for p_im in os.listdir('data/bucket2/ts')])

In [12]:
dice = []
diceOur = []

In [13]:
models = [[dice,mn_dice], [diceOur,mn_dice_our]]

In [14]:
for model in models:
    for patient in patients:
        for i in range(5): 
            model[0].append(getResultsIoU_accm(patient, model[1][i]))

In [15]:
headers = ['precision all','recall all','f-score all','precision big','recall big','f-score big','precision others','recall others','f-score others']

In [16]:
dice = np.array(dice)
diceOur = np.array(diceOur)

In [18]:
print('diceOur-dice:')
for i in range(len(headers)):
    print(headers[i])
    print(scipy.stats.ttest_rel(diceOur[:,i], dice[:,i]))

diceOur-dice:
precision all
TtestResult(statistic=1.825986930528597, pvalue=0.0755155882229485, df=39)
recall all
TtestResult(statistic=1.0274755589547666, pvalue=0.31052728777137917, df=39)
f-score all
TtestResult(statistic=2.087586254331233, pvalue=0.043416402729097574, df=39)
precision big
TtestResult(statistic=0.8901103996178161, pvalue=0.37886476034946237, df=39)
recall big
TtestResult(statistic=1.4236793154241283, pvalue=0.16249107378701308, df=39)
f-score big
TtestResult(statistic=1.0987628949435693, pvalue=0.2786085030456786, df=39)
precision others
TtestResult(statistic=1.9614107958379383, pvalue=0.05699831657490178, df=39)
recall others
TtestResult(statistic=0.5053366290092426, pvalue=0.6161665157984892, df=39)
f-score others
TtestResult(statistic=2.0873638971136184, pvalue=0.04343759632700639, df=39)
